In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")

In [16]:
from architectures.lstm import SimpleLSTM
from architectures.gpt import GPTDecoder

vocab_size = 50_000   # number of tokens
embed_dim = 384     # embedding dimension
hidden_dim = 384     # LSTM hidden size
num_layers = 2

lstm = SimpleLSTM(vocab_size, embed_dim, hidden_dim, num_layers)
param_count = sum(p.numel() for p in lstm.parameters() if p.requires_grad)
print(f"LSTM has {param_count} trainable params")

LSTM has 21615440 trainable params


In [17]:
vocab_size = 50_000
embed_dim = 256
num_heads = 8
ff_hidden_dim = 2048
num_layers = 6
context_length = 128
dropout = 0.1

gpt = GPTDecoder(
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    num_heads=num_heads,
    ff_hidden_dim=ff_hidden_dim,
    num_layers=num_layers,
    context_length=context_length,
    dropout=dropout
)

param_count = sum(p.numel() for p in gpt.parameters())
print(f"GPT has {param_count} trainable params")

GPT has 20690944 trainable params


In [4]:
import torch

def choose_device() -> str:
    if torch.cuda.is_available():
        return "cuda"
    elif torch.backends.mps.is_available():
        return "mps"
    else:
        return "cpu"

In [22]:
from torch.functional import F

@torch.no_grad()
def generate_text_lstm(model, tokenizer, prompt, max_new_tokens=20, device=None):
    if device is None:
        device = choose_device()
    
    model.to(device)
    model.eval()

    # Encode prompt
    tokens = tokenizer.encode(prompt, add_special_tokens=False)
    input_ids = torch.tensor(tokens, dtype=torch.long).unsqueeze(0).to(device)

    generated_tokens = tokens.copy()

    with torch.no_grad():
        hidden = None
        for _ in range(max_new_tokens):
            out, hidden = model(input_ids, hidden)
            last_logits = out[0, -1, :]  # last token
            probs = torch.softmax(last_logits, dim=-1)
            predicted_id = torch.argmax(probs).item()

            # Append predicted token
            generated_tokens.append(predicted_id)
            
            # Prepare next input
            input_ids = torch.tensor([[predicted_id]], dtype=torch.long).to(device)
    
    # Decode full sequence
    text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return text


@torch.no_grad()
def generate_text_gpt(model, tokenizer, prompt, max_new_tokens=20, device=None, temperature=1.2):
    if device is None:
        device = choose_device()

    model.to(device)
    model.eval()

    # Encode prompt
    tokens = tokenizer.encode(prompt, add_special_tokens=False)
    input_ids = torch.tensor(tokens, dtype=torch.long, device=device).unsqueeze(0)  # [1, T]

    for _ in range(max_new_tokens):
        # Forward pass through GPT
        logits = model(input_ids)  # shape: [1, seq_len, vocab_size]

        # Take the last token logits
        logits = logits / temperature
        logits = logits[0, -1, :]
        
        probs = F.softmax(logits, dim=-1)
        # print(probs.shape)
        next_token_id = torch.multinomial(probs, num_samples=1).reshape(1, 1)
        
        # Greedy decoding (argmax)
        # next_token_id = torch.argmax(probs).unsqueeze(0).unsqueeze(0)  # [1,1]

        # Append predicted token to sequence
        input_ids = torch.cat([input_ids, next_token_id], dim=1)

    # Decode full sequence
    generated_tokens = input_ids[0].tolist()
    text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return text


In [23]:
lstm.load_state_dict(torch.load("saved_models/lstm_final.pt", map_location=torch.device('cpu')))

<All keys matched successfully>

In [24]:
gpt.load_state_dict(torch.load("saved_models/gpt_final.pt", map_location=torch.device('cpu')))

<All keys matched successfully>

In [25]:
prompts = [
    "Czasem jedno słowo potrafi zmienić cały dzień.",
    "Wczoraj ktoś zostawił mi kartkę na ławce, bez podpisu.",
    "No dobra, ale kto w ogóle uznał, że to ma sens?",
    "To miało być tylko na chwilę, a wyszło jak zawsze.",
    "Nie wiem, czy to przez pogodę, czy przez ludzi, ale dziś wszystko wydaje się dziwnie ciche.",
    "„Nie klikaj tam” — powiedział, zanim ekran zgasł.",
    "W sumie nie planowałem o tym mówić, ale skoro już tu jesteś…",
    "Dwa dni bez snu i nagle wszystko zaczyna się układać. Ironia, co?",
    "Kiedy byłem mały, myślałem, że dorośli wszystko wiedzą.",
    "Czasami po prostu trzeba usiąść, włączyć coś spokojnego i udawać, że świat się nie pali."
]

In [29]:
generate_text_gpt(gpt, tokenizer, prompts[0], max_new_tokens=100, temperature=0.5)

'Czasem jedno słowo potrafi zmienić cały dzień . W każdym z nich jest to połączenie , a następnie jedno z dwóch możliwych znaków . W przypadku trzech znaków , które w każdym przypadku są zapisywane jako " user " , a nie " . W przypadku znaków , w których słowo " user " oznacza " stół " , jest to " stół " . Ozzzo – miejscowość i gmina we Włoszech , w regionie Piemont , w prowincji Turyn . Według danych na rok 2004 gminę zamieszkiwało 1101 osób , 15 os . / km² .'